In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import re
import nltk

from tensorflow.keras.layers import *
from sklearn.metrics import *
from transformers import BertTokenizer, TFBertModel
from gensim.utils import simple_preprocess
from math import ceil
import spektral

from politifact_bert import BigClass

In [2]:
train_df = pd.read_csv('./data/politifact-train-80-20.csv', index_col=0)
test_df = pd.read_csv('./data/politifact-test-80-20.csv', index_col=0)

self = BigClass(train_df, test_df, "Politifact", ['fake', 'real'])
self.prepare_dataset()
self.prepare_bert_tokens()
self.save_bert_tokens()
self.save_labels()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
D:\StudioPenguin\politifact\politifact_bert.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train_raw = self.raw_train_dataset.iloc[:, 1].replace(self.LABELS, [0, 1]).tolist()
D:\StudioPenguin\politifact\politifact_bert.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test_raw = self.raw_test_dataset.iloc[:, 1].replace(self.LABELS, [0, 1]).tolist()
D:\StudioPeng

Training Dataset -  label
real    401
fake    299
Name: count, dtype: int64
Train data shape -  (700, 2)
Test Dataset -  label
real    98
fake    77
Name: count, dtype: int64
Test data shape -  (175, 2)


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
input_word_ids = tf.keras.Input(shape=(self.MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.Input(shape=(self.MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.Input(shape=(self.MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_type_ids')

x = self.bert_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)
x = x[0]